In [1]:
import os
path_with_mask = r'C:\\Users\\Arshit\\Covid19 Mask Detection\\train\\with_mask'
path_without_mask = r'C:\\Users\\Arshit\\Covid19 Mask Detection\\train\\without_mask'

In [2]:
li_with_mask = os.listdir(path_with_mask)
li_without_mask = os.listdir(path_without_mask)

print(len(li_with_mask))
print(len(li_without_mask))

341
275


In [3]:
import cv2

In [4]:
X_train = []
for i in range(341):
    
    img = cv2.imread(path_with_mask+'\\'+li_with_mask[i])
    img = cv2.resize(img,(32,32))
    X_train.append(img)

In [5]:
for i in range(275):
    img = cv2.imread(path_without_mask+'\\'+li_without_mask[i])
    img = cv2.resize(img,(32,32))
    X_train.append(img)

In [6]:
import numpy as np

In [7]:
X_train = np.reshape(X_train,(len(X_train),)+(32,32,3))

In [8]:
Y_train = []
for i in range(len(li_with_mask)):
    Y_train.append(1)
    
for j in range(len(li_without_mask)):
    Y_train.append(0)

In [9]:
Y_train = np.reshape(Y_train,(len(Y_train),)+(1,))

In [10]:
print(X_train.shape)
print(Y_train.shape)

(616, 32, 32, 3)
(616, 1)


In [13]:
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)]
    return Y

In [14]:
Y_oh_train = convert_to_one_hot(Y_train, C = 2)

In [15]:
Y_oh_train.shape

(616, 2)

In [16]:
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np
from keras.layers import Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv2D, MaxPooling2D

Using TensorFlow backend.


In [17]:
import keras

In [18]:
model = keras.Sequential()

model.add(layers.Conv2D(filters=6, kernel_size=(3, 3), activation='tanh', input_shape=(32,32,3)))
model.add(layers.AveragePooling2D())

model.add(layers.Conv2D(filters=16, kernel_size=(3, 3), activation='tanh'))
model.add(layers.AveragePooling2D())

model.add(layers.Flatten())

model.add(layers.Dense(units=120, activation='relu'))

model.add(layers.Dense(units=84, activation='relu'))

model.add(layers.Dense(units=2, activation = 'softmax'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



In [19]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 30, 6)         168       
_________________________________________________________________
average_pooling2d_1 (Average (None, 15, 15, 6)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 16)        880       
_________________________________________________________________
average_pooling2d_2 (Average (None, 6, 6, 16)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 120)               69240     
_________________________________________________________________
dense_2 (Dense)              (None, 84)               

In [22]:
model.compile(loss=keras.losses.binary_crossentropy, optimizer='adam', metrics=['accuracy'])

In [23]:
model.fit(X_train, Y_oh_train, batch_size = 16, epochs=20)

Epoch 1/20
616/616 [==============================] - 0s 578us/step - loss: 0.2374 - accuracy: 0.9156
Epoch 2/20
616/616 [==============================] - 0s 360us/step - loss: 0.1382 - accuracy: 0.9464
Epoch 3/20
616/616 [==============================] - 0s 414us/step - loss: 0.1058 - accuracy: 0.9562
Epoch 4/20
616/616 [==============================] - 0s 329us/step - loss: 0.0635 - accuracy: 0.9740
Epoch 5/20
616/616 [==============================] - 0s 347us/step - loss: 0.0439 - accuracy: 0.9886
Epoch 6/20
616/616 [==============================] - 0s 327us/step - loss: 0.0370 - accuracy: 0.9903
Epoch 7/20
616/616 [==============================] - 0s 339us/step - loss: 0.0402 - accuracy: 0.9838
Epoch 8/20
616/616 [==============================] - 0s 317us/step - loss: 0.0373 - accuracy: 0.9838
Epoch 9/20
616/616 [==============================] - 0s 353us/step - loss: 0.0187 - accuracy: 0.9919
Epoch 10/20
616/616 [==============================] - 0s 347us/step - loss: 0.044

In [24]:
loss,acc = model.evaluate(X_train,Y_oh_train)

616/616 [==============================] - 0s 178us/step


In [25]:
predictions = model.predict(X_train)

In [26]:
print(loss,acc)

0.00045780193944334644 1.0


In [36]:
# test data

import os
path_with_mask = r'C:\\Users\\Arshit\\Covid19 Mask Detection\\test\\with_mask'
path_without_mask = r'C:\\Users\\Arshit\\Covid19 Mask Detection\\test\\without_mask'

li_with_mask = os.listdir(path_with_mask)
li_without_mask = os.listdir(path_without_mask)

print(len(li_with_mask))
print(len(li_without_mask))

X_test = []
for i in range(len(li_with_mask)):
    
    img = cv2.imread(path_with_mask+'\\'+li_with_mask[i])
    img = cv2.resize(img,(32,32))
    X_test.append(img)
    
for i in range(len(li_without_mask)):
    img = cv2.imread(path_without_mask+'\\'+li_without_mask[i])
    img = cv2.resize(img,(32,32))
    X_test.append(img)
    
X_test = np.reshape(X_test,(len(X_test),)+(32,32,3))
Y_test = []
for i in range(len(li_with_mask)):
    Y_test.append(1)
    
for j in range(len(li_without_mask)):
    Y_test.append(0)
    
Y_test = np.reshape(Y_test,(len(Y_test),)+(1,))

print(X_test.shape)
print(Y_test.shape)

Y_oh_test = convert_to_one_hot(Y_test, C = 2)

model.fit(X_test, Y_oh_test, batch_size = 16, epochs=20)

test_loss, test_acc = model.evaluate(X_test,Y_oh_test)

print(test_loss, test_acc)

66
132
(198, 32, 32, 3)
(198, 1)
Epoch 1/20
198/198 [==============================] - 0s 314us/step - loss: 1.5990e-04 - accuracy: 1.0000
Epoch 2/20
198/198 [==============================] - 0s 479us/step - loss: 1.5207e-04 - accuracy: 1.0000
Epoch 3/20
198/198 [==============================] - 0s 472us/step - loss: 1.4419e-04 - accuracy: 1.0000
Epoch 4/20
198/198 [==============================] - 0s 418us/step - loss: 1.3712e-04 - accuracy: 1.0000
Epoch 5/20
198/198 [==============================] - 0s 415us/step - loss: 1.3170e-04 - accuracy: 1.0000
Epoch 6/20
198/198 [==============================] - 0s 366us/step - loss: 1.2537e-04 - accuracy: 1.0000
Epoch 7/20
198/198 [==============================] - 0s 281us/step - loss: 1.2000e-04 - accuracy: 1.0000
Epoch 8/20
198/198 [==============================] - 0s 343us/step - loss: 1.1552e-04 - accuracy: 1.0000
Epoch 9/20
198/198 [==============================] - 0s 405us/step - loss: 1.1089e-04 - accuracy: 1.0000
Epoch 10/20
1

In [34]:
# Testing image without mask
random_image = cv2.imread('1.jpg')
random_image = cv2.resize(random_image,(32,32))
random_img = np.reshape(random_image,(1,)+(32,32,3))
pred = model.predict(random_img)
pred = np.argmax(pred)
if pred==1:
    print("Mask Detected")
else:
    print("Mask not detected")

Mask not detected


In [33]:
# testing image with mask
random_image = cv2.imread('79-with-mask.jpg')
random_image = cv2.resize(random_image,(32,32))
random_img = np.reshape(random_image,(1,)+(32,32,3))
pred = model.predict(random_img)
pred = np.argmax(pred)
if pred==1:
    print("Mask Detected")
else:
    print("Mask not detected")

Mask Detected
